<a href="https://colab.research.google.com/github/24p11/recode-scenario/blob/main/scenario_oncology_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create fictive clinical notes from Code set (DRG + ICD)

Code set are the raw classification data, we can extract from National database (Base nationale PMSI en France). They are made of 
* classification profile made of grouping variables from DRG records which are prepared with their frequency in the national database
    - age (class)
    - sexe
    - DRG (racine GHM)
    - Main diagnosis (ICD10) : cf
    - Hospitalization management type : cf
* diagnosis associated to each classification profile, extracted with their frequencies
* procedures associated to each classification profile, specialy for surgery and technical gestures, extracted with their frequencies

From thoses raw information we produce a coded clinical scenario which will be uses a seed.

This scenario is transformed into a detail prompt that will be given to a LLM for generation.
From the combinaision of primary and related diagnosis in French discharge abstract, we derived two notions :
* Primary diagnosis : host the notion of principal pathology, it is rather the primary diagnosis of the discharge abstract or the related diagnosis when it exists and that the primary diagnosis of the discharge abstract is from the chapter "Facteurs influant sur l’état de santé" of ICD10
* The Hospitalization management type is rather the term "Primary diagnosis" or the ICD-10 code of the related diagnosis when it exists


In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import pandas as pd
import numpy as np
import datetime as dt

In [7]:
from utils import *

In [5]:
gs = generate_scenario()
# Load official dictionaries
# col_names option allow you to algin your column names the project dictionary.
gs.load_offical_icd("cim_2024.xlsx",col_names={"code" : "icd_code","libelle":"icd_code_description"} )
gs.load_offical_procedures("ccam_actes_2024.xlsx",col_names={"code":"procedure","libelle_long":"procedure_description"} )
col_names={"Code CIM":"icd_parent_code","Localisation":"primary_site","Type Histologique":"histological_type",
	"Stade":"stage","Marqueurs Tumoraux":"biomarkers","Traitement":"treatment_recommandation","Protocole de Chimiothérapie":"chemotherapy_regimen"}
gs.load_cancer_treatement_recommandations("Tableau récapitulatif traitement cancer.xlsx",col_names ) 

In [6]:
# Load data from BN  PMSI
col_names={"racine":"drg_parent_code","das": "icd_secondary_code","diag":"icd_primary_code","categ_cim":"icd_primary_parent_code",
            "mdp":"case_management_type","nb_situations":"nb","acte":"procedure",
            "mode_entree":"admission_mode",
            "mode_sortie":"discharge_disposition",
            "mode_hospit":"admission_type"}
gs.load_classification_profile("bn_pmsi_cases_20250819.csv", col_names)
gs.load_secondary_icd("bn_pmsi_related_diag_20250818.csv",col_names)
gs.load_procedures("bn_pmsi_procedures_20250818.csv",col_names)

In [222]:
cols_scenario = ["first_name","last_name","cage2","cage","sexe",
                "last_name_med","icd_primary_code",
                "admission_type","admission_mode","discharge_disposition",
                'drg_parent_code','icd_primary_code','icd_secondaray_code','cd_md_pec']
                
cols_cancer = ["cancer_stage","TNM_score","histological_type","treatment_recommandation","chemotherapy_regimen"]



In [216]:
#Prepare cases
df_profile = gs.df_classification_profile.drop(columns="nb")
df_profile = df_profile[df_profile.icd_primary_code.isin(gs.icd_codes_cancer)]

In [236]:
current_profile["case_management_type"][0:1]

'D'

In [233]:
df_scenario =[]
for i in range(0,5):

    current_profile = df_profile.iloc[i,:]

    scenario = gs.generate_scenario_from_profile(current_profile)
    row = {k:scenario[k] for k in scenario if k in cols_scenario }
    cancer = [scenario[k] for k in scenario if k in cols_cancer ]

    row.update({"cancer":cancer})
    
    case  = gs.make_prompts_marks_from_scenario(scenario)
    
    row.update({'case': case})


    if row['admission_type'] == "Inpatient" and row['drg_parent_code'][2:3]=="C" :
        template_name = "surgery_complete.txt"
    elif row['admission_type'] == "Outpatient" and row['drg_parent_code'][2:3]=="C" :
        template_name = "surgery_outpatient.txt"
    elif row['drg_parent_code'][2:3]=="K" :
        template_name = "interventionnel.txt"
    elif row['cd_md_pec']==17 :
        template_name = "bilan.txt"
    else:
        template_name = "scenario_onco_v1.txt"
        
    prompt =  prepare_prompt("templates/" + template_name ,case =case)
    row.update({'prompt': prompt})

    df_scenario.append(row)

C:\Users\3056269\AppData\Roaming\Python\Python310\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
C:\Users\3056269\AppData\Roaming\Python\Python310\site-packages\pandas\core\indexing.py:1724: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer_missing(indexer, value)


In [226]:
df_scenario= pd.DataFrame(df_scenario)

In [227]:
df_scenario.to_csv(gs.path_data + "test_scenario_v1.csv")

In [212]:
df_scenario[]

{'sexe': 2,
 'first_name': 'Zeinab',
 'icd_secondaray_code': ['R2630', 'I10', 'C773', 'C795', 'C792', 'Z290'],
 'admission_mode': 'DOMICILE',
 'discharge_disposition': 'DOMICILE',
 'icd_primary_code': 'C50',
 'drg_parent_code': '28Z07',
 'cage': '[30-40[',
 'cage2': '[18-50[',
 'admission_type': 'Inpatient',
 'last_name': 'Jouhanet',
 'last_name_med': 'Lissillour',
 'cancer': ['Stade II',
  None,
  'Carcinome métaplasique',
  'Chirurgie (mastectomie ou BCS) + radiothérapie, thérapie systémique adjuvante incluant chimiothérapie, thérapie ciblée HER2 si HER2+',
  'TAC (Docetaxel, Doxorubicine, Cyclophosphamide)'],
 'case': {'SCENARIO': "**SCÉNARIO DE DÉPART :**\n- Âge du patient : 34ans\n- Sexe du patient : 2\n- Date d'entrée : 18/07/2024\n- Date de sortie : 20/07/2024\n- Date de naissance : 29/06/1990\n- Prénom du patient : Zeinab\n- Mode de prise en charge : Hospitalisation pour prise en charge du cancer\n- codage CIM10 :\n   * Diagnostic principal : Tumeur maligne du sein(C50)\n   * D

In [263]:
import yaml

# Chemin vers ton fichier YAML
fichier_yaml = "templates/regles_atih.yml"

# Ouverture et lecture du fichier
with open(fichier_yaml, "r", encoding="utf-8") as fichier:
    contenu = yaml.safe_load(fichier)

# Affichage du contenu
print(contenu)

{'regles': [{'id': 'D1', 'description': 'Règle D1', 'texte': "HOSPITALISATION POUR DIAGNOSTIC. La situation clinique de diagnostic est ainsi caractérisée par une démarche médicale de nature diagnostique. Celle-ci vise, à partir de la symptomatologie qui a motivé l’hospitalisation, à établir un diagnostic (suivi ou non d’un traitement).Ici le séjour a permis le diagnostic de l’affection causale. Le traitement a pu être réalisé au court de l'hospitalisation ou bien être remis à plus tard. Dans cette situation, le codage du DP utilise en général les chapitres I à XVII et XIX (voire XXII) de la CIM–10. Il ne fait pas appel aux « codes Z », il ne doit donc pas être mentionné de DR dans le RUM.\n", 'criteres': 'DP != "Z"'}, {'id': 'D2', 'description': 'Règle D2', 'texte': 'HOSPITALISATION POUR DIAGNOSTIC. La situation clinique de diagnostic est ainsi caractérisée par une démarche médicale de nature diagnostique. Celle-ci vise, à partir de la symptomatologie qui a motivé l’hospitalisation, à 

In [264]:
rules={}
for d in contenu["regles"] :
    rules[d["id"]] = {"texte": d["texte"], "criteres" : d["criteres"] }
rules

{'D1': {'texte': "HOSPITALISATION POUR DIAGNOSTIC. La situation clinique de diagnostic est ainsi caractérisée par une démarche médicale de nature diagnostique. Celle-ci vise, à partir de la symptomatologie qui a motivé l’hospitalisation, à établir un diagnostic (suivi ou non d’un traitement).Ici le séjour a permis le diagnostic de l’affection causale. Le traitement a pu être réalisé au court de l'hospitalisation ou bien être remis à plus tard. Dans cette situation, le codage du DP utilise en général les chapitres I à XVII et XIX (voire XXII) de la CIM–10. Il ne fait pas appel aux « codes Z », il ne doit donc pas être mentionné de DR dans le RUM.\n",
  'criteres': 'DP != "Z"'},
 'D2': {'texte': 'HOSPITALISATION POUR DIAGNOSTIC. La situation clinique de diagnostic est ainsi caractérisée par une démarche médicale de nature diagnostique. Celle-ci vise, à partir de la symptomatologie qui a motivé l’hospitalisation, à établir un diagnostic (suivi ou non d’un traitement). Ici il n’a pas été f

In [258]:
icd_primary_code = "R05"
icd_related_code = ""


'La symptomatologie qui a motivé l’hospitalisation et qui a été explorée, est le DP, qu’elle persiste ou qu’elle ait disparu lors du séjour. La situation clinique relative à la règle D2 est ainsi la seule exception à la consigne selon laquelle il ne peut être codé dans le RUM que des problèmes de santé présents (actifs) au moment de l’hospitalisation. Dans cette situation, le codage du DP utilise souvent le chapitre XVIII de la CIM–10 ; il ne fait pas appel aux codes Z, il ne doit donc pas être mentionné de DR dans le RUM. Sont équivalentes les situations rencontrées chez un patient atteint d’une affection chronique ou de longue durée connue, antérieurement diagnostiquée, admis pour une symptomatologie qui reste sans diagnostic étiologique. Le DP est la symptomatologie.\n'

In [262]:
gs.icd_code_chronic_attack = pd.read_csv("referentials/" + "icd_code_chronic_attack.txt",sep=";").code


In [ ]:
 def choose_atih_hierarchy_rule(icd_primary_code, icd_related_code, das=None, type_hosp=None, ds=None):
        """
        Détermine la situation clinique et la règle ATIH applicable en fonction des paramètres d'entrée.

        Args:
            selficd_primary_code (str): Diagnostic principal (code CIM-10).
            icd_related_code (str): Diagnostic relié (code CIM-10 ou None).
            das (list): Liste des diagnostics associés (codes CIM-10 ou None).
            type_hosp (str): "HC" (hospitalisation complète) ou "HP" (hospitalisation partielle).
            ds (int): Durée de séjour (en jours).

        Returns:
            dict: Un dictionnaire contenant la situation clinique et la règle ATIH applicable.
        """
        # Liste des règles et situations possibles
        situations = []

        # --- Règles liées à l'hospitalisation pour diagnostic ---
        # Règle D1 : DP = affection causale, DR = vide
        # On ajoute aussi la situation de pancréatite aigue, qui peut être un épdisode unique (règle D1) 
        # ou compliquer une pancréatite chronique (regle D5)

        icd_primary_description = gs.get_icd_description(icd_primary_code)

        if  ( not icd_primary_code.startswith('Z') and (icd_related_code is None) ) and icd_primary_code.startswith("K85"):
            
            situations.append({
                "situation":  rules["D2"]["texte"],
                "regle": "D1"
            })

        # Règle D2 : DP = symptomatologie, DR = None ou suspicion non confirmée
        if icd_primary_code.startswith('R') or icd_primary_code.startswith('Z03'):
            situations.append({
                "situation": "[Règles D2] " + rules["D2"]["texte"],
                "regle": "D2"
            })

        # Règle D3 : DP = Z04.800, Z04.801, Z04.802, Z01.5
        if icd_primary_code in ["Z04800", "Z04801", "Z04802", "Z015"]:
            situations.append({
                "situation": rules["D3"]["texte"] + icd_primary_description,
                "regle": "D3"
            })

        # Règle D4 : DP = motif de recours pour antécédent ou symptomatologie
        if icd_primary_code.startswith('Z8') or  icd_primary_code.startswith('Z9'):
            situations.append({
                "situation":  rules["D4"]["texte"] + icd_primary_description,
                "regle": "D4"
            })

        # Règle D5 : DP = maladie chronique en poussée aiguë
        if icd_primary_code in gs.icd_code_chronic_attack :
            situations.append({
                "situation":  rules["D5"]["texte"],
                "regle": "D5"
            })

        # Règle D6 : DP = code spécifique pour poussée aiguë (ex. : J44.1 pour BPCO)
        if icd_primary_code in gs.icd_code_acute_attack:
            situations.append({
                "situation": rules["D6"]["texte"],
                "regle": "D6"
            })

        # Règle D7 : DP = complication d’une maladie chronique
        if not icd_primary_code.startswith('Z') and (icd_related_code is not None and (icd_related_code.startswith('E') or icd_related_code.startswith('I') or icd_related_code.startswith('J') or icd_related_code.startswith('N'))):
            situations.append({
                "situation": "Séjour motivé par une complication d’une maladie chronique.",
                "regle": "D7"
            })

        # Règle D8 : DP = affection intercurrente indépendante
        if not icd_primary_code.startswith('Z') and (icd_related_code is not None and not icd_related_code.startswith('Z')):
            situations.append({
                "situation": "Séjour motivé par une affection intercurrente indépendante de la maladie chronique.",
                "regle": "D8"
            })

        # Règle D9 : DP = tumeur maligne (bilan initial d’extension)
        if icd_primary_code.startswith('C'):
            situations.append({
                "situation": "Séjour motivé par le bilan initial d’extension d’un cancer.",
                "regle": "D9"
            })

        # --- Règles liées à l'hospitalisation pour traitement ---
        # Règle T1 : DP = code Z pour traitement répétitif
        if icd_primary_code.startswith('Z') and icd_primary_code not in ["Z41.0", "Z41.1", "Z41.80", "Z43.0", "Z49.0", "Z51.01", "Z51.2", "Z51.5"]:
            situations.append({
                "situation": "Séjour pour traitement répétitif (DP = code Z).",
                "regle": "T1"
            })

        # Règle T2 : Exceptions à T1 (douleur chronique, ascite, épanchement pleural, toxine botulique)
        if icd_primary_code in ["R52.10", "R52.18", "R18", "J90", "J91", "J94"]:
            situations.append({
                "situation": f"Séjour pour traitement spécifique (DP = {icd_primary_code}).",
                "regle": "T2"
            })

        # Règle T3 : Traitement chirurgical unique
        if not icd_primary_code.startswith('Z') and type_hosp == "HC" and ds is not None and ds <= 7:
            situations.append({
                "situation": "Séjour pour traitement chirurgical unique (DP = maladie opérée).",
                "regle": "T3"
            })

        # Règle T4 : Chirurgie esthétique
        if icd_primary_code in ["Z41.0", "Z41.1"]:
            situations.append({
                "situation": "Séjour pour chirurgie esthétique (DP = code Z41).",
                "regle": "T4"
            })

        # Règle T5 : Chirurgie plastique non esthétique
        if icd_primary_code.startswith('Q') or icd_primary_code.startswith('S') or icd_primary_code.startswith('T') or icd_primary_code == "Z42":
            situations.append({
                "situation": "Séjour pour chirurgie plastique non esthétique (DP = code pathologique ou Z42).",
                "regle": "T5"
            })

        # Règle T6 : Intervention de confort
        if icd_primary_code == "Z41.80":
            situations.append({
                "situation": "Séjour pour intervention de confort (DP = Z41.80).",
                "regle": "T6"
            })

        # Règle T7 : Soins spécifiques de stomies/prothèses
        if icd_primary_code.startswith('Z4') and icd_primary_code not in ["Z41.0", "Z41.1", "Z41.80"]:
            situations.append({
                "situation": "Séjour pour soins spécifiques de stomies/prothèses (DP = code Z43 à Z47 ou Z49).",
                "regle": "T7"
            })

        # Règle T8 : Traitement interventionnel
        if not icd_primary_code.startswith('Z') and type_hosp == "HC" and ds is not None and ds <= 3:
            situations.append({
                "situation": "Séjour pour traitement interventionnel (DP = maladie traitée).",
                "regle": "T8"
            })

        # Règle T9 : Traitement médical unique
        if not icd_primary_code.startswith('Z') and type_hosp == "HC" and ds is not None and ds <= 5:
            situations.append({
                "situation": "Séjour pour traitement médical unique (DP = affection traitée).",
                "regle": "T9"
            })

        # Règle T10 : Curiethérapie ou injection de fer
        if icd_primary_code in ["Z51.01", "Z51.2"]:
            situations.append({
                "situation": "Séjour pour curiethérapie ou injection de fer (DP = code Z51).",
                "regle": "T10"
            })

        # Règle T11 : Soins palliatifs
        if icd_primary_code == "Z51.5":
            situations.append({
                "situation": "Séjour pour soins palliatifs (DP = Z51.5).",
                "regle": "T11"
            })

        # Règle T12 : Accouchement normal
        if icd_primary_code == "O80.0":
            situations.append({
                "situation": "Séjour pour accouchement normal (DP = O80.0).",
                "regle": "T12"
            })

        # Règle T13 : Nouveau-né en maternité
        if icd_primary_code.startswith('Z38'):
            situations.append({
                "situation": "Séjour pour nouveau-né en maternité (DP = code Z38).",
                "regle": "T13"
            })

        # Règle T14 : Reprise de traitement chirurgical
        if not icd_primary_code.startswith('Z') and type_hosp == "HC" and ds is not None and ds > 7:
            situations.append({
                "situation": "Séjour pour reprise de traitement chirurgical (DP = maladie traitée).",
                "regle": "T14"
            })

        # Règle T15 : Intervention prophylactique
        if icd_primary_code == "Z40":
            situations.append({
                "situation": "Séjour pour intervention prophylactique (DP = Z40).",
                "regle": "T15"
            })

        # --- Règles liées à l'hospitalisation pour surveillance ---
        # Règle S1 : Surveillance négative
        if icd_primary_code.startswith('Z') and icd_primary_code not in ["Z94", "Z95", "Z39.08", "Z76.2"]:
            situations.append({
                "situation": "Séjour pour surveillance négative (DP = code Z).",
                "regle": "S1"
            })

        # Règle S2 : Surveillance post-transplantation
        if icd_primary_code.startswith('Z94'):
            situations.append({
                "situation": "Séjour pour surveillance post-transplantation (DP = code Z94).",
                "regle": "S2"
            })

        # Règle S3 : Surveillance d’implant cardiovasculaire
        if icd_primary_code.startswith('Z95'):
            situations.append({
                "situation": "Séjour pour surveillance d’implant cardiovasculaire (DP = code Z95).",
                "regle": "S3"
            })

        # Règle SD1 : Surveillance positive
        if icd_primary_code.startswith('Z') and icd_related_code is not None and not icd_related_code.startswith('Z'):
            situations.append({
                "situation": "Séjour pour surveillance positive (DP = code Z, DR = affection nouvelle).",
                "regle": "SD1"
            })

        # Règle SD2 : Récidive de cancer
        if icd_primary_code.startswith('C') and icd_related_code is not None and icd_related_code.startswith('Z85'):
            situations.append({
                "situation": "Séjour pour récidive de cancer (DP = tumeur récidivante, DR = antécédent de cancer).",
                "regle": "SD2"
            })

        # Règle S4 : Surveillance post-accouchement
        if icd_primary_code in ["Z39.08", "Z76.2"]:
            situations.append({
                "situation": "Séjour pour surveillance post-accouchement ou nouveau-né sain (DP = code Z39 ou Z76).",
                "regle": "S4"
            })

        # --- Règles pour plusieurs DP possibles ---
        # Règle M1 : Un problème principal
        if len(situations) > 1:
            situations.append({
                "situation": "Plusieurs problèmes de santé, mais un problème principal mobilise l’essentiel des efforts de soins.",
                "regle": "M1"
            })

        # Règle M2 : Efforts équivalents
        if len(situations) > 1:
            situations.append({
                "situation": "Plusieurs problèmes de santé avec efforts de soins équivalents.",
                "regle": "M2"
            })

        # --- Tirage au sort si plusieurs situations possibles ---
        if len(situations) == 0:
            return {"situation": "Aucune règle ATIH ne correspond à ce scénario.", "regle": None}
        elif len(situations) == 1:
            return situations[0]
        else:
            return random.choice(situations)

    # Exemple d'utilisation
    if __name__ == "__main__":
        # Exemple 1 : DP = Z04.800, DR = G40.1
        print(choisir_regle_atih(icd_primary_code="Z04.800", icd_related_code="G40.1"))

        # Exemple 2 : DP = I21.9, DR = R07.4
        print(choisir_regle_atih(icd_primary_code="I21.9", icd_related_code="R07.4"))

        # Exemple 3 : DP = C50.9, DR = None
        print(choisir_regle_atih(icd_primary_code="C50.9", icd_related_code=None))

        # Exemple 4 : DP = Z51.5, DR = None
        print(choisir_regle_atih(icd_primary_code="Z51.5", icd_related_code=None))

In [15]:
df_res = pd.read_csv("results/test_generation_v1_response.csv")
    

In [16]:
df_res.columns


Index(['Unnamed: 0', 'age', 'sexe', 'date_entry', 'date_discharge',
       'date_of_birth', 'first_name', 'last_name', 'icd_primary_code',
       'case_management_type', 'icd_secondaray_code', 'admission_mode',
       'discharge_disposition', 'cancer_stage', 'score_TNM',
       'histological_type', 'treatment_recommandation', 'chemotherapy_regimen',
       'drg_parent_code', 'cage', 'cage2', 'admission_type', 'dms', 'los_mean',
       'los_sd', 'drg_parent_description', 'icd_parent_code',
       'icd_primary_description', 'case_management_type_description',
       'first_name_med', 'last_name_med', 'text_secondary_icd_official',
       'procedure', 'text_procedure', 'case_management_type_text', 'cd_md_pec',
       'prompt', 'biomarkers', 'response'],
      dtype='object')

In [20]:

df_res

In [34]:
i = 6
print(df_res.loc[i,"prompt"])
print(json.loads(df_res.loc[i,"response"] )['CR'])

Vous êtes un oncologue clinicien expert. Votre tâche est de générer un compte rendu d'hospitalisation en style clinique synthétique.


**SCÉNARIO DE DÉPART :**
- Âge du patient : 69 ans
- Sexe du patient : Féminin
- Date d'entrée : 10/01/2024
- Date de sortie : 10/01/2024
- Date de naissance : 26/07/1954
- Prénom du patient : Nazra
- Nom du patient : Velay
- Mode de prise en charge : Hospitalisation en ambulatoire pour Examen de contrôle après d'autres traitements pour tumeur maligne
- Codage CIM10 :
   * Diagnostic principal : Tumeur maligne du sein (C50)
   * Diagnostic relié : Examen de contrôle après d'autres traitements pour tumeur maligne (Z088)
   * Diagnostic associés : 
- Autres formes de schizophrénie (F208)
- Cachexie (R64)
- Tumeur maligne secondaire du foie et des voies biliaires intrahépatiques (C787)
- Tumeur maligne secondaire du cerveau et des méninges cérébrales (C793)
- Douleurs chroniques irréductibles, autres et non précisées (R5218)
- Pneumopathie bactérienne, san